In [167]:
%pip install gspread_pandas
%pip install fooddatacentral
%pip install pint
%pip install cufflinks

#Link to the deliverables https://github.com/ligonteaching/EEP153_Materials/blob/master/Project2/goals.pdf

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [168]:
#Configuration
from  scipy.optimize import linprog as lp
import numpy as np
import warnings
import pandas as pd
import eep153_tools
from eep153_tools.sheets import read_sheets
import fooddatacentral as fdc
import cufflinks as cf
cf.go_offline()
tol = 1e-5

apikey = 'XGLMhknvg8y65j7VpcMisC5nQrRGbdgqUe3GJRQY' 

#Test case to see if our APIKEY works
fdc.search(apikey,"marmite")
id =  2110211   # Put an FDC ID HERE!
fdc.nutrients(apikey,fdc_id=id)

,Quantity,Units
Total lipid (fat),3.53,g
"Vitamin C, total ascorbic acid",0.00,mg
"Vitamin A, IU",0.00,IU
Protein,8.24,g
"Carbohydrate, by difference",19.41,g
"Calcium, Ca",24.00,mg
Energy,141.00,kcal
"Sugars, total including NLEA",1.18,g
"Fatty acids, total trans",0.00,g
"Fatty acids, total saturated",0.88,g


In [169]:
#[A] Description of Population of Interest


In [170]:
#[A] Data on Prices for Different Foods 
# We explored the variability in prices for different foods sold off-campus and on-campus

#Here we are loading the dataframes with information we collected for off and on-campus foods 
#df is our on-campus dataframe, df_offcampus is our off-campus dataframe
df = read_sheets("1O1JwxbaDF1ubq_OY2wPdsjVH9Kcn1xollVqG8bmDpbU",sheet="on_campus") 
df_offcampus = read_sheets("1O1JwxbaDF1ubq_OY2wPdsjVH9Kcn1xollVqG8bmDpbU",sheet='off_campus')

df = df.set_index('Food')
df_offcampus = df_offcampus.set_index('Food')
df

Key available for students@eep153.iam.gserviceaccount.com.
Key available for students@eep153.iam.gserviceaccount.com.


,Quantity,Units,Price,Date,Location,FDC
Food,,,,,,
Turkey & Swiss Sandwich,225.0,grams,11.25,[2023-2-23],Cafe Think,2342427
Caprese Panini,225.0,grams,11.25,[2023-2-23],Cafe Think,468080
Chicken Paprika,225.0,grams,11.25,[2023-2-23],Cafe Think,1952920
Thai Marinated chicken Bahn Mi,241.0,grams,12.25,[2023-2-23],Cafe Think,2463885
French Fries,110.0,grams,4.75,[2023-2-23],Cafe Think,169833
...,...,...,...,...,...,...
Salad Seasonal Greens,190.0,grams,4.00,[2023-2-24],Brown's,2304126
Garlic Fries,100.0,grams,4.00,[2023-2-24],Brown's,1912834
Vegetable Medley,150.0,grams,4.00,[2023-2-24],Brown's,2133291


In [171]:
#Here we are cleaning the data by: 

# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))
df_offcampus['FDC Quantity'] = df_offcampus[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Dividing price by the FDC Quantity to get price per hectoliter
df['FDC Price'] = df['Price']/df['FDC Quantity']
df.dropna(how='any') # Drop food with any missing data
Prices = df.groupby('Food')['FDC Price'].min()

#Off campus calulations to transfer to hectoliter
df_offcampus['FDC Price'] = df_offcampus['Price']/df_offcampus['FDC Quantity']
df_offcampus.dropna(how='any') # Drop food with any missing data
Prices_off = df_offcampus.groupby('Food')['FDC Price'].min()


#These two variables show the prices per food off and on campus in hectoliters 
Prices 
Prices_off
Prices.to_string()

/opt/conda/envs/anaconda-2022.05-py39/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1981: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

/opt/conda/envs/anaconda-2022.05-py39/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1981: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



'Food\nAmericano                               1.0435213743417537 / hectogram\nBacon Burrito                                          3.5 / hectogram\nBacon Croissant                                        2.8 / hectogram\nBanana                                                 1.0 / hectogram\nBanana Muffin                           2.1666666666666665 / hectogram\nBeef Bourguignon                                     10.75 / hectogram\nBeyond Burger                            4.787234042553191 / hectogram\nBlack Beans & Corn Quesadilla                        5.475 / hectogram\nBreakfast Burrito                         3.58974358974359 / hectogram\nBreakfast Egg Croissant                  3.414634146341464 / hectogram\nBrown Rice                                             4.0 / hectogram\nBuffalo Chicken Loaded Fries                           4.0 / hectogram\nBuffalo Chicken Salad                                  4.0 / hectogram\nCalifornia Greens Salad                  4.28571428571428

In [172]:
#[A] Nutritional Content of Different Foods

#mapping food quantities to nutrients for df 
import warnings
D = {}
count = 0
for food in  df.index:
    try:
        FDC = df.loc[df.index==food,:].FDC.values[0]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
        print(food)
    except AttributeError:
        warnings.warn(f"Couldn't find FDC Code {FDC} for food {food}.")

D = pd.DataFrame(D,dtype=float)


#mapping food quantities to nutrients for df_offcampus
D_off = {}
count_off = 0
for food in df_offcampus.index:
    try:
        FDC_off = df_offcampus.loc[df_offcampus.index==food,:].FDC.values[0]
        count_off+=1
        D_off[food] = fdc.nutrients(apikey,FDC_off).Quantity
        print(food)
    except AttributeError:
        warnings.warn(f"Couldn't find FDC Code {FDC_off} for food {food}.")

D_off = pd.DataFrame(D_off,dtype=float)

#Following US nutrition reccomendations segmented by gender and age,
    # these are the max and min tables for on-campus and off-campus populations 
RDIs = read_sheets('https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/')
bmin = RDIs['diet_minimums'].set_index('Nutrition')
bmin = bmin.drop('Source',axis=1)

bmax = RDIs['diet_maximums'].set_index('Nutrition')
bmax = bmax.drop('Source',axis=1)


RDIs2 = read_sheets('https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/')
bmin_off = RDIs2['diet_minimums'].set_index('Nutrition')
bmin_off = bmin_off.drop('Source',axis=1)

bmax_off = RDIs2['diet_maximums'].set_index('Nutrition')
bmax_off = bmax_off.drop('Source',axis=1)

/tmp/ipykernel_797/624330623.py:14: UserWarning:

Couldn't find FDC Code 2300695 for food Salmon Larb Bowl.

/tmp/ipykernel_797/624330623.py:14: UserWarning:

Couldn't find FDC Code 2304126 for food Side Green Salad.

/tmp/ipykernel_797/624330623.py:14: UserWarning:

Couldn't find FDC Code 2304126 for food Salad Seasonal Greens.



Turkey & Swiss Sandwich
Caprese Panini
Chicken Paprika
Thai Marinated chicken Bahn Mi
French Fries
Black Beans & Corn Quesadilla 
Chicken Quesadilla 
Panko Crusted Fried Chicken Sandwich
Think Grass Fed Burger
Ham & Cheese Croissant
Banana
Chocolate Croissant
Banana Muffin
Tropical Hibusicus Green Tea
Drip Coffee (Reg or Decaf)
Espresso
Americano
Cappuccino
Latte
Caramel Macchiato
Hot Chocolate
Steamed Milk
Hot Tea
Chef Salad
Tropical Fruit Bowl
Raspberry Chobani Greek Yogurt 
Strawberry Banana Naked Juice 
Yerbamate
Winter Chopped Salad
Kale & Beets
Vegetable Cassoulet
Roast Chicken Breast
King Salmon & French Lentils
Beef Bourguignon
Chocolate Cake
Steamed Veggie Bun
Steamed Chicken Bun
Veggie Spring Rolls
Vietnamese Coffee
Vegan Pho 
Hibiscus Lemonade
Breakfast Burrito
Bacon Burrito
Turkey Sausage Burrito 
Breakfast Egg Croissant 
Bacon Croissant 
Turkey Saussage Croissant 
Hash Brown Patties
Small Fruit Cup
Philly Cheesesteak
Cheeseburger
Beyond Burger
Fried Chicken Sandwhich
Salmo

In [173]:
#Objective function to find the prices where quantities are all 100g
p = Prices.apply(lambda x:x.magnitude).dropna()
# Compile list that we have both prices and nutritional info for; drop if either missing
use = p.index.intersection(D.columns)
p = p[use]

#Maps bag of groceries x into nutrients, dropping nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[p.index].fillna(0)
# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]
Amax = Aall.loc[bmax.index]
# Maximum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin,-Amax])
b = pd.concat([bmin,-bmax])


#Objective function to find the prices where quantities are all 100g
p_off = Prices_off.apply(lambda x:x.magnitude).dropna()
# Compile list that we have both prices and nutritional info for; drop if either missing
use_off = p_off.index.intersection(D_off.columns)
p_off = p_off[use_off]

Aall_off = D_off[p_off.index].fillna(0)
Amin_off = Aall_off.loc[bmin.index]
Amax_off = Aall_off.loc[bmax.index]
# Maximum requirements involve multiplying constraint by -1 to make <=.
A_off = pd.concat([Amin_off,-Amax_off])
b_off = pd.concat([bmin_off,-bmax_off])

In [174]:
#[A] Dietary Reference Intakes 
#takes age and sex, returns pandas series of data reference intakes of a variety of nutrients 

#helper function
def helper(age, sex): 
    if (age < 4) or (sex == 'C'):
        group = 'C 1-3'
    elif age < 9: 
        group = str(sex) + ' 4-8'
    elif age < 14:
        group = str(sex) + ' 9-13'
    elif age < 19:
        group = str(sex) + ' 14-18'
    elif age < 31:
        group = str(sex) + ' 19-30'
    elif age < 51:
        group = str(sex) + ' 31-50'
    else: # over 51: 
        group = str(sex) + ' 51+'
    return group


def dietary_ref_index(age, sex):
    group = helper(age, sex)
    series = np.abs(b[group])
    return series 

dietary_ref_index(22, 'M') #this should return the Pandas Series

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Sodium, Na                        2300.0
Energy                            3100.0
Name: M 19-30, dtype: float64

In [175]:
#A Minimum Cost Diet Function 

groupM =  helper(22, 'M')
groupF =  helper(22, 'F')


resultM = lp(p, -A, -b[groupM], method='interior-point') #male on campus
resultF = lp(p, -A, -b[groupF], method='interior-point') #female on campus

result_Moff = lp(p_off, -A_off, -b[groupM], method='interior-point') #male off
result_Foff = lp(p_off, -A_off, -b[groupF], method='interior-point') #female off


#Print for the cost per day 
print(f"Cost of diet for {groupM} is ${resultM.fun:.2f} per day when dining on campus.") 
print(f"Cost of diet for {groupF} is ${resultF.fun:.2f} per day when dining on campus.")

print(f"Cost of diet for {groupM} is ${result_Moff.fun:.2f} per day off campus.")
print(f"Cost of diet for {groupF} is ${result_Foff.fun:.2f} per day off campus.")



Cost of diet for M 19-30 is $806.08 per day when dining on campus.
Cost of diet for F 19-30 is $410.80 per day when dining on campus.
Cost of diet for M 19-30 is $4.71 per day off campus.
Cost of diet for F 19-30 is $4.02 per day off campus.


In [176]:
#[A] Solution
#Print for what you will be eating 
dietfcamp = pd.Series(resultF.x,index=p.index)

#Females On-campus
print("\nFemales on campus will be eating (in 100s of grams or milliliters):")
print(dietfcamp[dietfcamp >= tol])  # Drop items with quantities less than precision of calculation.

#Males On-campus
dietmcamp = pd.Series(resultM.x,index=p.index)
print("\nMales on campus will be eating (in 100s of grams or milliliters):")
print(dietmcamp[dietmcamp >= tol]) 

#Females Off-campus
dietfoff = pd.Series(result_Foff.x,index=p_off.index)
print("\nFemales off campus will be eating (in 100s of grams or milliliters):")
print(dietfoff[dietfoff >= tol]) 

#Males Off-campus
dietmoff = pd.Series(result_Moff.x,index=p_off.index)
print("\nMales off campus will be eating (in 100s of grams or milliliters):")
print(dietmoff[dietmoff >= tol]) 


Females on campus will be eating (in 100s of grams or milliliters):
Bacon Burrito           0.433178
Bacon Croissant         0.067593
Banana                  0.279909
Banana Muffin           1.810406
Beef Bourguignon        0.246097
                          ...   
Vegetable Medley        2.669833
Veggie Spring Rolls     0.133996
Vietnamese Coffee       0.287027
Winter Chopped Salad    0.147617
Yerbamate               3.473749
Length: 70, dtype: float64

Males on campus will be eating (in 100s of grams or milliliters):
Bacon Burrito           0.399272
Bacon Croissant         0.081149
Banana                  0.264059
Banana Muffin           2.527383
Beef Bourguignon        0.216667
                          ...   
Vegetable Medley        3.155974
Veggie Spring Rolls     0.131625
Vietnamese Coffee       0.972422
Winter Chopped Salad    0.132325
Yerbamate               3.806071
Length: 70, dtype: float64

Females off campus will be eating (in 100s of grams or milliliters):
Black Beans   

In [185]:
#[B] Total Cost for population of interest 

total_undergrad = 45435
percent_berkeley_male = .45
percent_berkeley_female = .55

count_berkeley_male = percent_berkeley_male * total_undergrad
count_berkeley_female = percent_berkeley_female * total_undergrad

price_campus_male = count_berkeley_male * resultM.fun
price_campus_female = count_berkeley_female * resultF.fun

price_offcampus_male = count_berkeley_male * result_Moff.fun
price_offcampus_female = count_berkeley_female * result_Foff.fun

print(f"Total cost of diet for the Berkeley Male Population is ${price_campus_male} per day when dining on campus.") 
print(f"Total cost of diet for the Berkeley Female Population is ${price_campus_female} per day when dining on campus.") 

print(f"Total cost of diet for the Berkeley Male Population is ${price_offcampus_male} per day when dining off campus.") 
print(f"Total cost of diet for the Berkeley Female Population is ${price_offcampus_female} per day when dining off campus.") 


Total cost of diet for the Berkeley Male Population is $16480981.267620832 per day when dining on campus.
Total cost of diet for the Berkeley Female Population is $10265657.4127811 per day when dining on campus.
Total cost of diet for the Berkeley Male Population is $96394.99750372427 per day when dining off campus.
Total cost of diet for the Berkeley Female Population is $100546.92922820906 per day when dining off campus.


In [178]:
#[B] Is the Solution Edible ?


In [179]:
#[C] Sensitivity of Solution

#if the price of a particular kind of food increases by x%
    #how much does price of diet change and how does composition
    #of diet change? Vizualize these changes